In [1]:
from matplotlib import pyplot
import numpy as np
from helmi import Helmi

h = Helmi(5000)
h.run_circuits_on_Adonis()
h.run_circuits_on_Aer()
h.store_results()

/u/31/uotilav2/unix/Desktop/nanoGPT-for-error-mitigation/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Native operations of the backend: ['measure', 'id', 'r', 'cz', 'reset']
Coupling map of the backend: [[0, 2], [2, 0], [1, 2], [2, 1], [3, 2], [2, 3], [4, 2], [2, 4]]
Qubit list of the backend: 5


/u/31/uotilav2/data/Documents/karateclub/karateclub/estimator.py:66: UserWarning: Please do be advised that the graph you have provided does not contain (some) edges in the main diagonal, for instance the self-loop constitued of (0, 0). These selfloops are necessary to ensure that the graph is traversable, and for this reason we create a copy of the graph and add therein the missing edges. Since we are creating a copy, this will immediately duplicate the memory requirements. To avoid this double allocation, you can provide the graph with the selfloops.
  warnings.warn(


Validation context: (1000, 768)
Validation source: (1000,)
Validation target: (1000,)
Training context: (4000, 768)
Training source: (4000,)
Training target: (4000,)
Data stored successfully


In [2]:
if False:
    for i, (x1, x2) in enumerate(zip(expec1, expec2)):
        print(np.abs(x1, x2))
        
        if i < 10:
            fig = circuits[i].draw(output="mpl")
            display(fig)
            pyplot.close(fig)

In [3]:
if False:
    import rustworkx as rx
    import networkx as nx

    def dag_node_to_dict(dag_node):
        return {
            'successors': dag_node.successors if dag_node.successors is not None else [],
            'predecessors': dag_node.predecessors if dag_node.predecessors is not None else [],
            'matchedwith': dag_node.matchedwith if dag_node.matchedwith is not None else [],
            'successorstovisit': dag_node.successorstovisit if dag_node.successorstovisit is not None else [],
            'isblocked': dag_node.isblocked if dag_node.isblocked is not None else False,
            'qindices': dag_node.qindices if dag_node.qindices is not None else [],
            'cindices': dag_node.cindices if dag_node.cindices is not None else []
        }

    def rustworkx_to_networkx(rx_graph: rx.PyDiGraph) -> nx.DiGraph:
        # Create a new directed graph
        nx_graph = nx.DiGraph()

        # Add nodes to the NetworkX graph
        for node in rx_graph.node_indices():
            dag_node = rx_graph[node]
            nx_graph.add_node(node, feature = dag_node_to_dict(dag_node))

        # Add edges to the NetworkX graph
        for edge in rx_graph.edge_list():
            edge_data = rx_graph.get_edge_data(edge[0], edge[1])
            nx_graph.add_edge(edge[0], edge[1], **edge_data)

        return nx_graph

In [4]:
if False:
    from qiskit import QuantumCircuit
    from qiskit.converters import circuit_to_dagdependency
    from karateclub.graph_embedding import Graph2Vec # type: ignore
    from rustworkx.visualization import mpl_draw

    # Create a Quantum Circuit acting on the q register
    circ = QuantumCircuit(2, 2)
    circ.h(0)
    circ.cx(0, 1)
    circ.measure_all()

    dag = circuit_to_dagdependency(circ)
    rxgraph = dag.to_retworkx()
    nxgraph = rustworkx_to_networkx(rxgraph)

    # The default values except attributed=True
    vectorizer = Graph2Vec(wl_iterations = 2, 
                        attributed = True, 
                        dimensions = 128, 
                        workers = 8, 
                        down_sampling = 0.0001, 
                        epochs = 10, 
                        learning_rate = 0.025, 
                        min_count = 1, 
                        seed = 42, 
                        erase_base_features = False)
    vectorizer.fit([nxgraph])
    vectors = vectorizer.get_embedding()
    print(vectors)